### Data pre-processing and data augmentation

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array,array_to_img
from keras.applications.xception import Xception,preprocess_input

Using TensorFlow backend.


In [3]:
datagen=ImageDataGenerator(rotation_range=40,
                          width_shift_range=0.2,
                          height_shift_range=0.2,
                          shear_range=0.2,
                          zoom_range=0.2,
                          horizontal_flip=True,
                          fill_mode='nearest')

In [15]:
os.chdir("E:\Python for MLAI\JIGSAW\\7-Deep Learning Algorithms\CNN Projects analyticsvidhya\Mini Project for cat and do classification\\train\\cats")

In [17]:
img=load_img("cat.2.jpg")
img=img_to_array(img)
img=np.expand_dims(img,axis=0)
img=preprocess_input(img)

In [19]:
i=0
for batch in datagen.flow(img,batch_size=1,save_to_dir='preview',save_prefix='cat',save_format='jpeg'):
    i+=1
    if i>20:
        break

In [7]:
def preprocess(im):
    im=img_to_array(im)
    im=np.expand_dims(im,axis=0)
    im=preprocess_input(im)
    return im[0]

In [8]:
data_dir="E:\Python for MLAI\\JIGSAW\Projects\\Project 7 Cat vs Dog Classification DL"

In [9]:
os.chdir(os.path.join(data_dir,"train",'cats'))
cat_train_img_names=glob.glob("*.jpg")

In [10]:
cat_train=[]
for i in cat_train_img_names:
    im=load_img(i,target_size=(224,224,3))
    cat_train.append(preprocess(im))

In [11]:
os.chdir(os.path.join(data_dir,"train",'dogs'))
dogs_train_img_names=glob.glob("*.jpg")

In [12]:
dog_train=[]
for i in dogs_train_img_names:
    im=load_img(i,target_size=(224,224,3))
    dog_train.append(preprocess(im))

In [49]:
os.chdir(os.path.join(data_dir,"test",'cat'))
cat_test_img_names=glob.glob("*.jpg")

In [50]:
cat_test=[]
for i in cat_test_img_names:
    im=load_img(i,target_size=(224,224,3))
    cat_test.append(preprocess(im))

In [51]:
os.chdir(os.path.join(data_dir,"test",'dog'))
dog_test_img_names=glob.glob("*.jpg")

In [52]:
dog_test=[]
for i in dog_test_img_names:
    im=load_img(i,target_size=(224,224,3))
    dog_test.append(preprocess(im))

In [55]:
cat_test=np.array(cat_test,dtype='float32')
cat_train=np.array(cat_train,dtype='float32')
dog_test=np.array(dog_test,dtype='float32')
dog_train=np.array(dog_train,dtype='float32')

In [58]:
cat_test.shape,cat_train.shape,dog_test.shape,dog_train.shape

((400, 224, 224, 3),
 (1000, 224, 224, 3),
 (400, 224, 224, 3),
 (1000, 224, 224, 3))

In [59]:
train=np.concatenate((cat_train,dog_train),axis=0)
test=np.concatenate((cat_test,dog_test),axis=0)

In [60]:
train.shape

(2000, 224, 224, 3)

In [61]:
test.shape

(800, 224, 224, 3)

In [63]:
y_train=[0]*cat_train.shape[0]+[1]*dog_train.shape[0]
y_test=[0]*cat_test.shape[0]+[1]*dog_test.shape[0]

In [76]:
y_train=np.array(y_train,dtype='float32')
y_test=np.array(y_test,dtype='float32')

In [77]:
base_model=Xception(include_top=False,pooling='avg',input_shape=(224,224,3))
train_features=base_model.predict(train)

In [78]:
train_features.shape

(2000, 2048)

In [79]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()

In [83]:
clf.fit(train_features,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [84]:
test_features=base_model.predict(test)

In [85]:
test_features.shape

(800, 2048)

In [86]:
preds=clf.predict(test_features)

In [87]:
(preds==y_test).sum()/y_test.shape[0]

0.985